In [1]:
#

In [1]:
import json
import time
import numpy
import pandas
from scipy import stats
from data.flat import load
from matplotlib import pyplot
from matplotlib import lines as mlines

E:\venv\financial_news_re\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer
from sklearn.linear_model import LinearRegression

In [5]:
from m_utils.measures import r2_adj
from m_utils.sampling import ts_sampler
from m_utils.summarization import get_ols_summary
from m_utils.feature_selection import recursive_batch

In [7]:
def load_data(data, tsi_names, y_names, removes, test_rate, n_folds):

    # clarify names

    exclude = y_names + removes
    x_names = [x for x in data.columns.values if (x not in exclude and 'LAG0' not in x)]
    
    # sample (without folds)

    data_train, data_test = ts_sampler(data, n_folds, test_rate)
    X_, Y_ = numpy.array(x_names), numpy.array(y_names)
    X_train, Y_train = [x[X_].values for x in data_train], [x[Y_].values for x in data_train]
    X_test, Y_test = data_test[X_].values, data_test[Y_].values
    
    print('Fold shape: {0}'.format(X_train[0].shape))
    print('Test shape: {0}'.format(X_test.shape))
    
    return X_train, Y_train, X_test, Y_test, X_
    

In [8]:
def censory(model, XX, YY):
    if XX.shape[1] == 0:
        mask = numpy.array([False])
    else:
        p_values = get_ols_summary(model, XX, YY, list(range(XX.shape[1])))['Probabilities'].values[1:]
        mask = p_values < 0.05
    return mask

In [9]:
def golags(tsi_names, y_names, removes, test_rate=0.2, n_folds=1):
    report = pandas.DataFrame(columns=['R2_adj', 'd1', 'X_adj_'])
    X_train, Y_train, X_test, Y_test, X_ = load_data(data, tsi_names, y_names, removes, test_rate, n_folds)
    params = {'n_jobs': -1, 'fit_intercept': False}
    model = LinearRegression
    for j in range(len(X_train)):
        
        model_adj, X_train_adj, Y_train_adj, X_test_adj, Y_test_adj, X_adj_ = recursive_batch(model, params, X_train[j], Y_train[j], censory, X_test, Y_test, X_)
        Y_hat_train_adj = model_adj.predict(X_train_adj)
        Y_hat_test_adj = model_adj.predict(X_test_adj)
        result = {'R2_adj': r2_adj(Y_train_adj, Y_hat_train_adj, X_train_adj.shape[0], X_train_adj.shape[1]), 
                  'd1': X_train_adj.shape[1], 'X_adj_': X_adj_}
        report = report.append(result, ignore_index=True)

    return report

In [10]:
d = './dataset.csv'
data = pandas.read_csv(d)
data = data.rename(columns={'lag': 'news_horizon'})
data = data.set_index(['ticker', 'time', 'news_horizon'], drop=False)
data = data.sort_index()

tsi_names = ['news_time']
y_names = ['open_LAG0']
removes = ['ticker', 'time', 'id', 'title', 'news_time']

X_ = golags(tsi_names, y_names, removes, n_folds=5)

Fold shape: (21483, 501)
Test shape: (36978, 501)


E:\venv\financial_news_re\lib\site-packages\m_utils\summarization.py:22: RuntimeWarning: invalid value encountered in sqrt
  sd_b = numpy.sqrt(var_b)
E:\venv\financial_news_re\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
E:\venv\financial_news_re\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
E:\venv\financial_news_re\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0
<ipython-input-8-e985140f5040>:6: RuntimeWarning: invalid value encountered in less
  mask = p_values < 0.05
E:\venv\financial_news_re\lib\site-packages\m_utils\summarization.py:22: RuntimeWarning: invalid value encountered in sqrt
  sd_b = numpy.sqrt(var_b)
E:\venv\financial_news_re\lib\site-packages\scipy\stats\_distn_infrastructur

In [12]:
X_

,R2_adj,d1,X_adj_
0,0.601345,115,"[volume_LAG2, open_LAG3, high_LAG3, volume_LAG..."
1,0.999784,231,"[close_LAG1, high_LAG1, open_LAG2, low_LAG2, v..."
2,0.987352,216,"[low_LAG1, open_LAG2, low_LAG2, volume_LAG2, o..."
3,0.685673,115,"[low_LAG1, volume_LAG1, open_LAG3, close_LAG3,..."
4,0.956138,125,"[open_LAG1, close_LAG1, low_LAG1, close_LAG2, ..."
